<a href="https://colab.research.google.com/github/Nastya880/cuda/blob/main/lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Лабораторная 3**

Редукция

In [ ]:
%%writefile reduction.cu
#include <stdio.h>
#include <stdint.h>
#include <assert.h>
#include <cuda_runtime.h>
#include "device_launch_parameters.h"
#include <iostream>
#include <time.h>
#include <random>

const int BLOCK_SIZE = 16;
const int GRID_SIZE = 16;

const int arr_length = BLOCK_SIZE*GRID_SIZE;

const dim3 threads(BLOCK_SIZE, 1);
const dim3 grid(GRID_SIZE, 1);

//__global__ void reduction(float * inData, float * outData)
//{
//	__shared__ int data[BLOCK_SIZE];
//	int tid = threadIdx.x;
//	int i = 2 * blockIdx.x * blockDim.x + threadIdx.x;
//	data[tid] = inData[i] + inData[i + blockDim.x];
//	__syncthreads();
//	for (int s = blockDim.x / 2; s > 0; s >>= 1)
//	{
//		if (tid < s)
//			data[tid] += data[tid + s];
//		__syncthreads();
//	}
//	if (tid == 0)
//		outData[blockIdx.x] = data[0];
//}

__global__ void reduction(float * inData, float * outData)
{
	__shared__ int data[BLOCK_SIZE];
	int tid = threadIdx.x;
	int i = 2 * blockIdx.x * blockDim.x + threadIdx.x;
	data[tid] = inData[i] < inData[i + blockDim.x]? inData[i] : inData[i + blockDim.x];
	__syncthreads();
	for (int s = blockDim.x / 2; s > 0; s >>= 1)
	{
		if (tid < s)
			data[tid] = data[tid] < data[tid + s] ? data[tid] : data[tid + s];
		__syncthreads();
	}
	if (tid == 0)
		outData[blockIdx.x] = data[0];
}

void generateRandom(float *arr) {
	for (int i = 0; i < arr_length; i++)
	{
		//arr[i] = 1;
		arr[i] = rand() % 40 + 10;
	}
}

void showArray(float *arr)
{
	for (int i = 0; i < arr_length; i++)
	{
		printf_s("%.0f ", arr[i]);
	}
	printf_s("\n\n");
}

void checkCudaErrors(cudaError_t err)
{
	if (err != cudaSuccess)
	{
		printf("%s\n", cudaGetErrorString(err));
		system("pause");
	}
}

void cpu_reduction(float* arr, int size) {
	double time = 0;
	float min = 999999;
	clock_t start = clock();
	for (int i = 0; i < size; i++)
		if (min > arr[i])
			min = arr[i];
	clock_t end = clock();
	printf("CPU result %.2f\n", min);
	printf("CPU time: %.5f ms\n", (double)(end - start) / CLOCKS_PER_SEC * 1000);
}

// Start the main CUDA Sample here
int main(int argc, char **argv)
{
	int memorySize = sizeof(float) * arr_length;

	//Выделение памяти на хосте
	float *host_A = (float*)malloc(memorySize);
	float *host_C = (float*)malloc(memorySize);

	//Выделение памяти на девайсе
	checkCudaErrors(cudaSetDevice(0));
	float *device_A;
	float *device_C;
	checkCudaErrors(cudaMalloc((void**)&device_A, memorySize));
	checkCudaErrors(cudaMalloc((void**)&device_C, memorySize));


	//Инициализация данных и отображение
	generateRandom(host_A);
	showArray(host_A);

	//Копирование данных на девайс
	checkCudaErrors(cudaMemcpy(device_A, host_A, memorySize, cudaMemcpyHostToDevice));


	//Настройка и запуск ядра
	cudaEvent_t start, stop;
	float gpuTime = 0.0f;
	checkCudaErrors(cudaEventCreate(&start));
	checkCudaErrors(cudaEventCreate(&stop));
	checkCudaErrors(cudaEventRecord(start, 0));

	reduction <<<grid, threads >>> (device_A, device_C);

	checkCudaErrors(cudaEventRecord(stop, 0));
	checkCudaErrors(cudaEventSynchronize(stop));
	checkCudaErrors(cudaEventElapsedTime(&gpuTime, start, stop));
	checkCudaErrors(cudaEventDestroy(start));
	checkCudaErrors(cudaEventDestroy(stop));

	//Копирование данных с девайса
	checkCudaErrors(cudaMemcpy(host_C, device_C, memorySize, cudaMemcpyDeviceToHost));

	//Проверка результатов и освобождение памяти
	showArray(host_C);
	printf("GPU time: %.5f ms\n", gpuTime);
	float result = 99999;
	for (int i = 0; i < arr_length; i++)
		if (result > host_C[i] && host_C[i] > 0)
			result = host_C[i];
	printf("GPU result: %.2f \n", result);
	cpu_reduction(host_A, arr_length);

	cudaDeviceReset();
	cudaFree(device_A);
	cudaFree(device_C);
	free(host_A);
	free(host_C);

	system("pause");
	return 0;
}


In [ ]:
!nvcc reduction.cu -o reduction -Wno-deprecated-gpu-targets
!nvprof ./reduction

гистограмма 

In [ ]:
#include <stdio.h>
#include <stdint.h>
#include <assert.h>
#include <cuda_runtime.h>
#include "device_launch_parameters.h"
#include <iostream>
#include <time.h>
#include <random>

typedef unsigned int uint;
typedef unsigned char uchar;

#define N				(6*1024*1024)	//Размер массива

#define LOG2_WARP_SIZE  5				// логарифм размера warp's по основанию 2
#define WARP_SIZE       32				// Размер warp'а
#define TAG_MASK		0x07FFFFFFU		//Маска для снятия идентификатора нити
#define NUM_BINS		256				// число счетчиков в гистограмме 
#define NUM_WARPS       6				// Число warp'ов в блоке 
#define MERGE_THREADBLOCK_SIZE	256
inline __device__ void addByte(volatile uint * warpHist, uint data, uint threadTag)
{
	uint count;
	do {
		// прочесть текущее значение счетчика и снять идентификатор нити 
		count = warpHist[data] & TAG_MASK;
		
		// увеличить его на единицу и поставить свой идентификатор
		count = threadTag | (count + 1);

		//осуществить запись
		warpHist[data] = count;
	}
	while (warpHist[data] != count); // проверить, прошла ли запись
}

inline __device__ void addWord(volatile uint *warpHist, uint data, uint tag)
{
	addByte(warpHist, (data >> 0) & 0xFFU, tag);
	addByte(warpHist, (data >> 8) & 0xFFU, tag);
	addByte(warpHist, (data >> 16) & 0xFFU, tag);
	addByte(warpHist, (data >> 24) & 0xFFU, tag);
}

__global__ void histogramKernel(uint * partialHistograms, uint * data, uint dataCount)
{
	//Своя гистограмма на каждый варп
	__shared__ uint hist[NUM_BINS*NUM_WARPS];
	uint *warpHist = hist + (threadIdx.x >> LOG2_WARP_SIZE) * NUM_BINS;

	// очистить счетчики гистограмм
#pragma unroll
	for (uint i = 0; i < NUM_BINS / WARP_SIZE; i++)
		hist[threadIdx.x + i*NUM_WARPS*WARP_SIZE] = 0;

	// получить id для данной нити
	uint tag = threadIdx.x << (32 - LOG2_WARP_SIZE);

	__syncthreads();

	//Построить гистограммы по заданному набору элементов
	for (uint pos = blockIdx.x * blockDim.x + threadIdx.x;
		pos < dataCount;
		pos += blockDim.x * gridDim.x)
	{
		uint d = data[pos];
		addWord(warpHist, d, tag);
	}
	__syncthreads();

	// объединить гистограммы данного блока и записать результат в глобальную память
	// 192 нити суммируют данные до 256 элементов гистограммы
	for (uint bin = threadIdx.x; bin < NUM_BINS; bin += (NUM_WARPS*WARP_SIZE))
	{
		uint sum = 0;
		for (uint i = 0; i < NUM_WARPS; i++)
			sum += hist[bin + i*NUM_BINS] & TAG_MASK;

		partialHistograms[blockIdx.x * NUM_BINS + bin] = sum;
	}
}

// объединить гистограммы, один блок на каждый NUM_BINS элементов
__global__ void mergeHistogramKernel(uint * outHistogram, uint * partialHistograms, uint histogramCount)
{
	uint sum = 0;
	for (uint i = threadIdx.x; i < histogramCount; i += 256)
		sum += partialHistograms[blockIdx.x + i * NUM_BINS];

	__shared__ uint data[NUM_BINS];

	data[threadIdx.x] = sum;

	for (uint stride = NUM_BINS / 2; stride > 0; stride >>= 1)
	{
		__syncthreads();

		if (threadIdx.x < stride)
			data[threadIdx.x] += data[threadIdx.x + stride];
	}

	if (threadIdx.x == 0) 
		outHistogram[blockIdx.x] = data[0];
}

void histogram(uint * histogram, void * dataDev, uint byteCount)
{
	assert(byteCount % 4 == 0);

	int n = byteCount / 4;
	int numBlocks = n / (NUM_WARPS*WARP_SIZE);
	int numPartials = 240;
	uint *partialHistograms = NULL;

	//выделить память под гистограммы блоков
	cudaMalloc((void**)&partialHistograms, numPartials*NUM_BINS * sizeof(uint));
	
	// построить гистограмму для каждого блока
	histogramKernel <<<dim3(numPartials), dim3(NUM_WARPS*WARP_SIZE) >>> (partialHistograms, (uint*)dataDev, n);
	
	//объдинить гистограммы отдельных блоков вместе
	mergeHistogramKernel <<<dim3(NUM_BINS), dim3(256) >>> (histogram, partialHistograms, numPartials);
	
	// освободить выделенную память
	cudaFree(partialHistograms);
}
//Заполнить массив случайными байтами
void randomInt(uint *a, int n, uint *h)
{
	std::random_device rd;
	std::mt19937 gen(rd());
	std::normal_distribution<> X(128, 10);
	double seconds = 0;
	for (int i = 0; i < n; i++)
	{
		uchar b1 = static_cast<int>(X(gen)) & 0xFF;
		uchar b2 = static_cast<int>(X(gen)) & 0xFF;
		uchar b3 = static_cast<int>(X(gen)) & 0xFF;
		uchar b4 = static_cast<int>(X(gen)) & 0xFF;

		a[i] = b1 | (b2 << 8) | (b3 << 16) | (b4 << 24);


		clock_t start = clock();


		h[b1]++;
		h[b2]++;
		h[b3]++;
		h[b4]++;
		clock_t end = clock();
		seconds += (double)(end - start) / CLOCKS_PER_SEC;

	}
	printf("CPU time: %.2f ms\n", seconds*1000);
}



int main()
{
	uint *a = new uint[N];
	uint *hDev = NULL;
	uint *aDev = NULL;
	uint h[NUM_BINS];
	uint hHost[NUM_BINS];
	cudaEvent_t start, stop;
	float gpuTime = 0.0f;

	memset(hHost, 0, sizeof(hHost));
	randomInt(a, N, hHost);

	cudaEventCreate(&start);
	cudaEventCreate(&stop);
	cudaEventRecord(start, 0);

	cudaMalloc((void**)&aDev, N * sizeof(uint));
	cudaMalloc((void**)&hDev, NUM_BINS * sizeof(uint));
	cudaMemcpy(aDev, a, N * sizeof(uint), cudaMemcpyHostToDevice);

	histogram(hDev, aDev, N * 4);

	cudaMemcpy(h, hDev, NUM_BINS * sizeof(uint), cudaMemcpyDeviceToHost);
	cudaFree(aDev);
	cudaFree(hDev);

	cudaEventRecord(stop, 0);
	cudaEventSynchronize(stop);
	cudaEventElapsedTime(&gpuTime, start, stop);

	printf("GPU time: %.2f ms\n", gpuTime);
	for (int i = 0; i < NUM_BINS; i++)
	{
		//if (h[i] != hHost[i])
			printf("Diff at %d-%d, %d\n",i,h[i],hHost[i]);
	}
	delete a;
	

	system("pause");
	return 0;
}
